In [8]:
from collections import Counter
import numpy as np
from nltk.tokenize import word_tokenize as tokenize

In [9]:
from setup import get_train, get_valid, get_test

In [10]:
train = get_train(6)
valid = get_valid(6)
test = get_test(6)

In [11]:
train.head(2)

,ID,label,statement,subject,speaker,speaker's job,state info,party,barely true counts,false counts,half true counts,mostly true counts,pants on fire counts,venue,lying ratio,ratio significance
0,2635.json,1,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,1.0,0,1,0,0,0,a mailer,0.8,0.01
1,10540.json,3,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,0.0,0,0,1,1,0,a floor speech.,0.3,0.02


In [12]:
#import from parents directory
import os, sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))

import utils

In [13]:
GLOVE_SHAPE = 50

In [14]:
#get pre-trained glove 
GLOVE_HOME = os.path.join('..','data', 'glove.6B')
glove = utils.glove2dict(os.path.join(GLOVE_HOME, 'glove.6B.' + str(GLOVE_SHAPE) + 'd.txt'))

# Job

So far, I've seen 1108 jobs. But some seem to be duplicates

'member of the u.s. house' and 'u.s. representative'

also, some are highly specific 
'u.s. representative, florida district 2'
could be interesting to test if we can jut remove everythign 

later idea: choose x amount (say 50) amount of the most common words in the set, that aren't fluff words, and strip out all the words that aren't in that set, 

anotehr idea: strip out everythign after the comma (e.g. 'ceo, apple inc', state senator, district 32)
another idea: a network to figure out what's important in the metadata

another idea: Since sentence order doesn't matter as much, BERT embeddings matter less.

In [15]:
#function to analyze the most common jobs
def most_common_job():
    job = train["speaker's job"]
    total = []
    for i, j in enumerate(job):
        if type(j) is not float:
            total.append(j.strip())
    c = Counter(total)
    most = c.most_common()
    return most

parse_job()

NameError: name 'parse_job' is not defined

## Average Glove Embeddings
### And play with different dimension sizes. 
If time, can experiment with tf-idf weighting of glove embeddings



In [ ]:
def get_embeddings(col):
    total = []
    for idx, s in enumerate(col):
        avg = np.zeros((1, GLOVE_SHAPE)) #initialize avg
        
        if type(s) is not float: 
            tokenized = tokenize(s)
            for t in tokenized:
                t = t.lower()
                if t in glove: #ignore if not in glove dictionary
                    emb = glove.get(t).reshape(1, -1)
                    avg += emb
            if not len(tokenized):
                assert False, 'job at index {} is not valid'.format(idx)
            avg /= len(tokenized)
            
        total.append(avg) #if NaN (no job listed), embedding is a 0 array
            
    return np.concatenate(total, axis = 0)

In [77]:
train_embeddings = get_embeddings(train["speaker's job"])

In [78]:
train_embeddings.shape #shape is (#setences, #embedding size)

(10241, 50)

In [84]:
valid_embed = get_embeddings(valid["speaker's job"])

In [85]:
valid_embed.shape #shape is (#setences, #embedding size)

(1284, 50)

In [86]:
test_embed = get_embeddings(test["speaker's job"])

In [92]:
test_embed[0][5] #shape is (#setences, #embedding size)

-0.28077

In [109]:

from job import get_most_common

ImportError: cannot import name 'get_most_common' from 'job' (/Users/mauriciow/Documents/CS/224.nosync/224ULieDetect/job.py)

In [104]:
get_job_valid().shape

(1284, 50)

# Venue